In [ ]:
pip install tensorflow opencv-python

In [ ]:
#train_data_directory = 'https://drive.google.com/drive/folders/16cEKULnazJsSyPzIZCSo2I6xjcCr80FL?usp=drive_link'

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
def extract_box_count(filename):
    parts = filename.lower().split('_')
    for part in parts:
        if 'boxes' in part:
            try:
                return int(part)
            except ValueError:
                pass
    return 0  

In [ ]:
def create_counting_model(input_shape=(224, 224, 3)):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='linear')) 
    return model

In [ ]:
model = create_counting_model()
model.compile(optimizer='adam', loss='mean_squared_error')  

In [ ]:
train_data_directory = '/content/drive/MyDrive/M_P Dataset csv/Inventorypictures'
#https://drive.google.com/drive/folders/16cEKULnazJsSyPzIZCSo2I6xjcCr80FL?usp=drive_link

In [ ]:
epochs = 5

for epoch in range(epochs):
    for filename in os.listdir(train_data_directory):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(train_data_directory, filename)

            image = cv2.imread(image_path)
            image = cv2.resize(image, (224, 224))
            image = image / 255.0 

            image = np.expand_dims(image, axis=0)
            count = np.array([[extract_box_count(filename)]])  

            model.train_on_batch(image, count)

In [ ]:
model.save('/content/drive/MyDrive/M_P Dataset csv/trained_model.h5')

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model('/content/drive/MyDrive/M_P Dataset csv/trained_model.h5')

In [ ]:
def count_boxes(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  
    image = image / 255.0  
    image = np.expand_dims(image, axis=0)  

    prediction = model.predict(image)
    count = int(np.round(prediction[0][0]))

    return count

In [ ]:
image_path = '/content/drive/MyDrive/M_P Dataset csv/Inventorypictures/20231025_145833.jpg'
box_count = count_boxes(image_path)
print(f'The predicted box count is: {box_count}')

1/1 [==============================] - 0s 44ms/step
The predicted box count is: 0


In [ ]:
image_path = '/content/drive/MyDrive/M_P Dataset csv/Inventorypictures/20231025_145833.jpg'
box_count = count_boxes(image_path)
print(f'The predicted box count is: {box_count}')

1/1 [==============================] - 0s 49ms/step
The predicted box count is: 0


In [ ]:
def count_boxes(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224)) 
    image = image / 255.0  
    image = np.expand_dims(image, axis=0) 

    prediction = model.predict(image)

    raw_prediction = prediction[0]
    print(f'Raw prediction values: {raw_prediction}')

    count = int(np.round(raw_prediction))

    return count

image_path = '/content/drive/MyDrive/M_P Dataset csv/Inventorypictures/IMG-20231103-WA0049.jpeg'
box_count = count_boxes(image_path)
print(f'The predicted box count is: {box_count}')

1/1 [==============================] - 0s 75ms/step
Raw prediction values: [0.00244552]
The predicted box count is: 0


In [ ]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection import model_lib_v2

pipeline_config_path = '/content/drive/MyDrive/M_P Dataset csv/pipeline.config'
model_dir = ''

configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
model_config = configs['model']


config_util.save_pipeline_config(configs, pipeline_config_path)

print("Training started...")
model_lib_v2.train_loop(
    pipeline_config_path=pipeline_config_path,
    model_dir=model_dir,
    train_steps=your_train_steps,  
    use_tpu=False,  
    checkpoint_every_n=your_checkpoint_frequency, 
)

print("Training finished.")


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

model = tf.saved_model.load('/content/drive/MyDrive/M_P Dataset csv/trained_model.h5')    

def count_boxes_faster_rcnn(image_path):
    image = cv2.imread(image_path)
    image_np = np.expand_dims(np.asarray(image, dtype=np.uint8), 0)

    input_tensor = tf.convert_to_tensor(image_np, dtype=tf.float32)

    detections = model(input_tensor)

    box_count = detections['num_detections'][0].numpy()

    return int(box_count)

In [ ]:
image_path = 'path/to/your/image.jpg'
box_count = count_boxes_faster_rcnn(image_path)
print(f'The predicted box count is: {box_count}')